# 🚀 KKday 크롤러 v1.0
## 통합 상품 데이터 수집 시스템

### 📋 주요 기능:
- ✅ **서울 페이지 진입으로 봇 탐지 회피**
- ✅ **검색박스 활용한 도시별 크롤링**
- ✅ **중앙화 셀렉터 시스템** (fallback 지원)
- ✅ **개별 파서 함수** (복잡한 속성 처리)
- ✅ **강화된 에러 처리** 및 재시도 메커니즘
- ✅ **24개 CSV 컬럼** 완전 지원
- ✅ **이미지 다운로드** 및 경로 관리
- ✅ **URL 중복 처리** 및 순위 연속성 보장

### 🔥 **v1.0 핵심 특징:**
- **KKdayCrawler 클래스**: 통합 크롤링 시스템
- **중앙화 + 개별 파서**: 최적 하이브리드 아키텍처
- **더미 테스트 검증**: 100% 로직 검증 완료
- **TimeoutException 처리**: 안정적인 대기 시스템
- **StaleElement 처리**: DOM 변화 대응

### 🎯 사용법:
1. **아래 1번 셀에서 도시명 및 목표 수량 설정**
2. **Run All 실행** (완전 자동화)
3. **결과 분석** (자동 통계 생성)

In [ ]:
# ===== 🎯 사용자 설정 영역 =====

# 1. 크롤링할 도시명 입력
CITY_NAME = "방콕"   #🔥🔥 도시명 입력 🔥🔥

# 2. 수집할 상품 수 설정
TARGET_PRODUCTS = 2  # 수집할 상품 수

# 3. 크롤링 범위 설정
MAX_PAGES = 3  # 최대 검색할 페이지 수

# 4. 이미지 저장 여부
SAVE_IMAGES = True  # True: 이미지 다운로드, False: URL만 저장

print("="*70)
print("🚀 KKday 크롤러 v1.0 시작")
print("="*70)

# ===== 환경 설정 및 모듈 Import =====
import sys
import os
import time
import json
from datetime import datetime

# 현재 kkday 폴더에서 src 폴더에 접근
sys.path.append('./src')
sys.path.append('.')

# KKday 프로젝트 모듈 import
try:
    from src.scraper.crawler import KKdayCrawler
    from src.config import CONFIG
    from src.utils.file_handler import create_product_data_structure, ensure_directory_structure
    print("✅ KKday 모듈 로드 성공")
except ImportError as e:
    print(f"❌ KKday 모듈 로드 실패: {e}")
    print("💡 src/ 폴더 구조를 확인하세요.")
    raise

# 의존성 확인
try:
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    print("✅ Selenium 모듈 로드 성공")
except ImportError:
    print("❌ Selenium이 설치되지 않았습니다.")
    print("💡 해결: pip install selenium")
    raise

# ===== 설정 검증 =====
print("\n📋 크롤링 설정:")
print(f"   🏙️ 도시: {CITY_NAME}")
print(f"   🎯 목표 상품: {TARGET_PRODUCTS}개")
print(f"   📄 최대 페이지: {MAX_PAGES}개")
print(f"   📸 이미지 저장: {'✅' if SAVE_IMAGES else '❌'}")

# 디렉토리 구조 확보
try:
    ensure_directory_structure(CITY_NAME)
    print(f"   📁 디렉토리 구조 확보 완료")
except Exception as e:
    print(f"   ⚠️ 디렉토리 구조 확보 실패: {e}")

print("\n🎯 설정 완료 - 크롤링 시작 준비!")
print("💡 진행상황은 실시간으로 표시됩니다.")

In [ ]:
# ===== 🚀 메인 크롤링 실행 =====
print(f"🚀 '{CITY_NAME}' 크롤링 시작!")
print("="*70)

# 크롤링 시작 시간 기록
start_time = datetime.now()
print(f"⏰ 시작 시간: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# KKday 크롤러 생성 및 실행
crawler = None
crawling_success = False

try:
    # 1. KKday 크롤러 초기화
    print(f"\n🏗️ KKday 크롤러 초기화...")
    crawler = KKdayCrawler(city_name=CITY_NAME)
    
    # 2. 전체 크롤링 실행
    print(f"\n🎯 전체 크롤링 실행 (최대 {MAX_PAGES}페이지, {TARGET_PRODUCTS}개 상품)")
    crawling_success = crawler.run_full_crawling(
        max_pages=MAX_PAGES,
        max_products=TARGET_PRODUCTS
    )
    
    if crawling_success:
        print("\n🎉 크롤링 성공적으로 완료!")
    else:
        print("\n⚠️ 크롤링이 완료되었지만 일부 문제가 있을 수 있습니다.")
        
except KeyboardInterrupt:
    print("\n⏹️ 사용자가 크롤링을 중단했습니다.")
    crawling_success = False
    
except Exception as e:
    print(f"\n❌ 크롤링 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    crawling_success = False

finally:
    # 크롤링 종료 시간 기록
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n⏰ 종료 시간: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️ 총 소요 시간: {duration}")
    
    # 크롤러 통계 출력 (크롤러가 있는 경우)
    if crawler and hasattr(crawler, 'stats'):
        print(f"\n📊 크롤링 통계:")
        stats = crawler.stats
        print(f"   • 전체 처리: {stats.get('total_processed', 0)}개")
        print(f"   • 성공: {stats.get('success_count', 0)}개")
        print(f"   • 실패: {stats.get('error_count', 0)}개")
        print(f"   • 건너뜀: {stats.get('skip_count', 0)}개")
        
        if stats.get('total_processed', 0) > 0:
            success_rate = (stats.get('success_count', 0) / stats.get('total_processed', 1)) * 100
            print(f"   • 성공률: {success_rate:.1f}%")

print(f"\n{'='*70}")
print(f"🏁 크롤링 실행 단계 완료")
print(f"{'='*70}")

In [ ]:
# ===== 📊 크롤링 결과 분석 =====
print(f"📊 '{CITY_NAME}' 크롤링 결과 분석")
print("="*70)

import os
import pandas as pd
# 'src.config'에서 도시 정보를 가져오는 함수를 import 합니다.
from src.config import get_city_location

try:
    # 1. 크롤링 상태 조회
    print("\n🔍 크롤링 상태 조회...")
    print("   (상태 요약은 이전 셀의 실행 결과를 참고하세요)")

    # 2. CSV 파일 분석
    print("\n📋 CSV 데이터 분석...")

    csv_path = None
    df = None
    csv_files_found = False
    expected_csv_path = "경로 미설정"  # 로그 출력을 위한 변수

    try:
        continent, country = get_city_location(CITY_NAME)
        is_city_state = CITY_NAME == country or CITY_NAME in ["홍콩", "싱가포르", "마카오", "괌"]

        if is_city_state:
            # 도시국가: 대륙 직하에 통합 파일
            expected_csv_path = os.path.join("data", continent, f"{CITY_NAME}_통합_kkday_products.csv")
            if os.path.exists(expected_csv_path):
                csv_path = expected_csv_path
        else:
            # 일반 국가: 국가 폴더 아래에 통합 파일
            expected_csv_path = os.path.join("data", continent, country, f"{country}_통합_kkday_products.csv")
            if os.path.exists(expected_csv_path):
                csv_path = expected_csv_path
            else:
                # 폴백: 도시별 파일 확인 (통합 파일이 생성 안됐을 경우)
                fallback_path = os.path.join("data", continent, country, CITY_NAME, f"kkday_{CITY_NAME}_products.csv")
                if os.path.exists(fallback_path):
                    csv_path = fallback_path
                    expected_csv_path = fallback_path  # 로그 출력을 위해 경로 업데이트

    except Exception as e:
        print(f"   ⚠️ CSV 경로 결정 중 오류: {e}")

    if csv_path and os.path.exists(csv_path):
        try:
            print(f"   📄 분석할 CSV 파일: {csv_path}")
            df = pd.read_csv(csv_path, encoding='utf-8-sig')
            csv_files_found = True
            print(f"\n📈 데이터 통계:")
            print(f"   • 총 상품 수: {len(df)}개")
            print(f"   • CSV 컬럼 수: {len(df.columns)}개")
            print(f"   • 파일 크기: {os.path.getsize(csv_path):,} bytes")

            essential_fields = ['상품명', '가격', '평점', 'URL', '순위']
            print(f"\n✅ 필수 필드 완성도:")
            for field in essential_fields:
                if field in df.columns:
                    if len(df) > 0:
                        valid_count = len(df[df[field].notna() & (df[field] != '') & (df[field] != '정보 없음')])
                        completion_rate = (valid_count / len(df)) * 100
                        status = "✅" if completion_rate >= 80 else "⚠️" if completion_rate >= 50 else "❌"
                        print(f"   {status} {field}: {completion_rate:.1f}% ({valid_count}/{len(df)})")
                    else:
                        print(f"   - {field}: 데이터 없음")

        except ImportError:
            print("   ℹ️ pandas가 없어 상세 분석을 건너뜁니다.")
        except Exception as e:
            print(f"   ❌ CSV 파일 분석 실패: {e}")
            df = None
            csv_files_found = False
    else:
        print(f"   ⚠️ CSV 파일을 찾을 수 없습니다. (예상 경로: {expected_csv_path})")

    # 3. 이미지 파일 확인
    print(f"\n🖼️ 이미지 파일 확인...")
    image_dir = None
    images_found = False
    expected_img_dir = "경로 미설정"  # 로그 출력을 위한 변수

    try:
        continent, country = get_city_location(CITY_NAME)
        is_city_state = CITY_NAME == country

        if is_city_state:
            image_dir = os.path.join("kkday_img", continent, country)
        else:
            image_dir = os.path.join("kkday_img", continent, country, CITY_NAME)
        expected_img_dir = image_dir

    except Exception as e:
        print(f"   ⚠️ 이미지 경로 결정 중 오류: {e}")

    if image_dir and os.path.exists(image_dir):
        image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        images_found = len(image_files) > 0
        print(f"   📸 저장된 이미지: {len(image_files)}개")
        if images_found:
            total_size = sum(os.path.getsize(os.path.join(image_dir, f)) for f in image_files)
            print(f"   💾 총 크기: {total_size/1024/1024:.2f} MB")
            main_images = [f for f in image_files if '_thumb' not in f]
            thumb_images = [f for f in image_files if '_thumb' in f]
            print(f"   🖼️ 메인 이미지: {len(main_images)}개")
            print(f"   🔍 썸네일: {len(thumb_images)}개")
    else:
        print(f"   📸 이미지 디렉토리 없음. (예상 경로: {expected_img_dir})")

    # 4. 크롤링 품질 평가
    print(f"\n🎯 크롤링 품질 평가:")
    quality_score = 0
    max_score = 5

    if 'crawling_success' in locals() and crawling_success:
        quality_score += 1
        print(f"   ✅ 크롤링 실행: 성공")
    else:
        print(f"   ❌ 크롤링 실행: 실패")

    if csv_files_found:
        quality_score += 1
        print(f"   ✅ CSV 생성: 성공")
    else:
        print(f"   ❌ CSV 생성: 실패")

    if df is not None and not df.empty:
        quality_score += 1
        print(f"   ✅ 데이터 수집: {len(df)}개 상품")

        essential_completion = 0
        for field in ['상품명', '가격', '평점']:
            if field in df.columns:
                if len(df) > 0:
                    valid_count = len(df[df[field].notna() & (df[field] != '')])
                    if (valid_count / len(df) >= 0.8):
                        essential_completion += 1

        if essential_completion >= 2:
            quality_score += 1
            print(f"   ✅ 데이터 품질: 우수")
        else:
            print(f"   ⚠️ 데이터 품질: 보통")
    else:
        print(f"   ❌ 데이터 수집: 실패")

    if SAVE_IMAGES and images_found:
        quality_score += 1
        print(f"   ✅ 이미지 저장: 성공")
    elif not SAVE_IMAGES:
        quality_score += 1
        print(f"   ✅ 이미지 설정: URL만 저장 (설정대로)")
    else:
        print(f"   ❌ 이미지 저장: 실패")

    quality_percentage = (quality_score / max_score) * 100
    status_emoji = "🎉 우수" if quality_percentage >= 80 else "👍 양호" if quality_percentage >= 60 else "⚠️ 보통" if quality_percentage >= 40 else "❌ 미흡"
    print(f"\n🏆 종합 품질 점수: {quality_score}/{max_score} ({quality_percentage:.1f}%) {status_emoji}")

except Exception as e:
    print(f"❌ 결과 분석 중 오류: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'='*70}")
print(f"📊 결과 분석 완료")
print(f"{'='*70}")

In [ ]:
# ===== 🎉 크롤링 완료 요약 =====
print(f"🎉 KKday 크롤러 v1.0 실행 완료")
print("="*70)

# 최종 실행 요약
print(f"📋 실행 요약:")
print(f"   🏙️ 크롤링 도시: {CITY_NAME}")
print(f"   🎯 목표 상품: {TARGET_PRODUCTS}개")
print(f"   📄 최대 페이지: {MAX_PAGES}개")
print(f"   📸 이미지 저장: {'활성화' if SAVE_IMAGES else '비활성화'}")

# 실행 시간
if 'start_time' in locals() and 'end_time' in locals():
    print(f"   ⏱️ 실행 시간: {end_time - start_time}")

# 성공/실패 상태
if crawling_success:
    print(f"   ✅ 크롤링 상태: 성공")
else:
    print(f"   ⚠️ 크롤링 상태: 부분 성공 또는 실패")

# 다음 단계 안내
print(f"\n💡 다음 단계:")
print(f"   1️⃣ 수집된 CSV 데이터 확인 및 검토")
print(f"   2️⃣ 이미지 파일 품질 확인 (다운로드된 경우)")
print(f"   3️⃣ 데이터 후처리 및 분석")
print(f"   4️⃣ 다른 도시 크롤링 (CITY_NAME 변경 후 재실행)")

# 파일 위치 안내
print(f"\n📁 생성된 파일 위치:")
print(f"   📄 CSV: data/{CITY_NAME}/")
print(f"   🖼️ 이미지: images/{CITY_NAME}/")
print(f"   📊 랭킹: ranking_data/ (해당하는 경우)")

# 문제 해결 안내
if not crawling_success:
    print(f"\n🔧 문제 해결:")
    print(f"   • 인터넷 연결 확인")
    print(f"   • Chrome 브라우저 및 ChromeDriver 버전 확인")
    print(f"   • 방화벽/보안 프로그램 설정 확인")
    print(f"   • 다른 도시명으로 테스트 시도")
    print(f"   • TARGET_PRODUCTS를 더 적은 수로 설정")

print(f"\n🚀 KKday 크롤러를 이용해 주셔서 감사합니다!")
print(f"={'*'*70}")

# 간단한 통계 출력 (가능한 경우)
if 'crawler' in locals() and crawler and hasattr(crawler, 'stats'):
    stats = crawler.stats
    success_count = stats.get('success_count', 0)
    total_processed = stats.get('total_processed', 0)
    
    if total_processed > 0:
        print(f"\n🎊 최종 성과: {success_count}/{total_processed} 상품 성공적으로 수집!")
        success_rate = (success_count / total_processed) * 100
        print(f"📈 성공률: {success_rate:.1f}%")
        
        if success_rate >= 80:
            print(f"🎉 우수한 성과입니다!")
        elif success_rate >= 60:
            print(f"👍 양호한 결과입니다!")
        else:
            print(f"💪 다음번엔 더 좋은 결과를 위해 설정을 조정해보세요!")